In [1]:
pip install python-binance


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from binance.client import Client
import pandas as pd
import numpy as np

In [3]:
client = Client()


In [4]:
def fetch_binance_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1HOUR, lookback="2 days ago UTC"):
    klines = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume", 
        "close_time", "quote_asset_volume", "number_of_trades", 
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    df = df[["timestamp", "open", "high", "low", "close", "volume"]].astype(float)
    return df


In [5]:
import gym
from gym import spaces

class BinanceTradingEnv(gym.Env):
    def __init__(self, symbol="BTCUSDT"):
        self.symbol = symbol
        self.data = fetch_binance_data(symbol)
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # 0: Hold, 1: Buy, 2: Sell
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32)

    def reset(self):
        self.data = fetch_binance_data(self.symbol)
        self.current_step = 0
        self.position = 0  # 1 = long, -1 = short, 0 = flat
        self.entry_price = 0
        return self._get_obs()

    def _get_obs(self):
        row = self.data.iloc[self.current_step]
        return np.array([row.open, row.high, row.low, row.close, row.volume], dtype=np.float32)

    def step(self, action):
        current_price = self.data.iloc[self.current_step].close
        reward = 0

        # Simulate trade
        if action == 1 and self.position == 0:  # Buy
            self.position = 1
            self.entry_price = current_price
        elif action == 2 and self.position == 0:  # Sell
            self.position = -1
            self.entry_price = current_price
        elif action == 0 and self.position != 0:  # Exit position
            reward = (current_price - self.entry_price) * self.position
            self.position = 0
            self.entry_price = 0

        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)

        return obs, reward, done, {}


In [6]:
from stable_baselines3 import PPO

env = BinanceTradingEnv(symbol="BTCUSDT")
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\sneh_\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 47       |
|    ep_rew_mean     | 361      |
| time/              |          |
|    fps             | 91       |
|    iterations      | 1        |
|    time_elapsed    | 22       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 47           |
|    ep_rew_mean          | 289          |
| time/                   |              |
|    fps                  | 84           |
|    iterations           | 2            |
|    time_elapsed         | 48           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0055185673 |
|    clip_fraction        | 0.0811       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -1.91e-06    |
|    learning_r